In [516]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

In [517]:
df = pd.read_csv('data.csv')

In [518]:
df.head(5)

,id,order_id,store_name,order_date,created_at,cook_time,name,is_topping,quantity
0,1642757,1642757,Brunch & Munch - Cartagena,2021-06-30 08:57:14,2021-06-30 14:13:21.934158,16:07,Grilled Cheese Sandwich,False,1
1,1642757,1642757,Brunch & Munch - Cartagena,2021-06-30 08:57:14,2021-06-30 14:13:21.683716,16:07,Grilled Cheese Sandwich,False,1
2,1642757,1642757,Brunch & Munch - Cartagena,2021-06-30 08:57:14,2021-06-30 14:13:21.934158,16:07,No Cubiertos,True,1
3,1642757,1642757,Brunch & Munch - Cartagena,2021-06-30 08:57:14,2021-06-30 14:13:21.683716,16:07,No Cubiertos,True,1
4,1642757,1642757,Brunch & Munch - Cartagena,2021-06-30 08:57:14,2021-06-30 14:13:21.934158,16:07,No Cubiertos,True,1


In [519]:
print(df['id'].count())
print(df.shape)

2387935
(2387935, 9)


## Buscamos la tienda con mayor cantidad de registros

In [520]:
%config IPCompleter.greedy = True

In [521]:
df.groupby('store_name').agg('count').sort_values(by='id',ascending=False).head(3)

,id,order_id,order_date,created_at,cook_time,name,is_topping,quantity
store_name,,,,,,,,
Brunch & Munch Medellin,51229,51229,51229,51229,51229,51229,51229,51229
La Cuadra - Bucaramanga,44373,44373,44373,44373,44373,44373,44373,44373
Burritos & Co Salitre,43364,43364,43364,43364,43364,43364,43364,43364


## Se trabajara con la tienda Brunch & Munch Medellin

In [522]:
df_store = df[df['store_name'] == 'Brunch & Munch Medellin']

In [523]:
df_store.shape

(51229, 9)

In [524]:
# Delete duplicated rows

df_store = df_store.drop_duplicates(subset=None, 
                          keep='first', 
                          inplace=False, 
                       )

In [525]:
df_store['id'].count()

46082

In [526]:
df_store.columns

Index(['id', 'order_id', 'store_name', 'order_date', 'created_at', 'cook_time',
       'name', 'is_topping', 'quantity'],
      dtype='object')

### Ajustar nombres

In [527]:
old_names = ['id', 'order_id', 'store_name', 'order_date', 'created_at', 
       'name', 'is_topping', 'quantity', 'cook_time'] 
new_names = ['id', 'order_id', 'store_name', 'order_date', 'on_hand', 
       'product', 'is_topping', 'quantity', 'cook_time']
df_store.rename(columns = dict(zip(old_names, new_names)), inplace = True)

In [528]:
# select features of interest for model
df_store = df_store[['order_id','order_date', 'on_hand','product', 'quantity', 'cook_time']]

In [529]:
df_store.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 46082 entries, 833 to 2386288
Data columns (total 6 columns):
order_id      46082 non-null int64
order_date    46082 non-null object
on_hand       46082 non-null object
product       46082 non-null object
quantity      46082 non-null int64
cook_time     46082 non-null object
dtypes: int64(2), object(4)
memory usage: 2.5+ MB


In [530]:
# Validate null values

df_store.isnull().sum()

order_id      0
order_date    0
on_hand       0
product       0
quantity      0
cook_time     0
dtype: int64

In [531]:
df_store.head(5)

,order_id,order_date,on_hand,product,quantity,cook_time
833,1644738,2021-06-30 21:40:16,2021-07-01 02:58:08.51549,Limonada,1,17:52
834,1644738,2021-06-30 21:40:16,2021-07-01 02:58:08.51549,Pancakes,1,17:52
835,1644738,2021-06-30 21:40:16,2021-07-01 02:58:08.51549,No Cubiertos,1,17:52
836,1644738,2021-06-30 21:40:16,2021-07-01 02:58:08.51549,Porción Hash Browns,1,17:52
837,1644738,2021-06-30 21:40:16,2021-07-01 02:58:08.51549,Egg Bacon & Cheese Croissant,1,17:52


###  Ajustamos el tipo de datos de las columnas


In [532]:
df_store['order_date'] = pd.to_datetime(df_store['order_date']).dt.to_period('T').dt.to_timestamp()
df_store['on_hand'] = pd.to_datetime(df_store['on_hand']).dt.to_period('T').dt.to_timestamp()

In [533]:
df_store['on_hand'] = df_store['on_hand'] - timedelta(hours=5)

In [534]:
df_store.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 46082 entries, 833 to 2386288
Data columns (total 6 columns):
order_id      46082 non-null int64
order_date    46082 non-null datetime64[ns]
on_hand       46082 non-null datetime64[ns]
product       46082 non-null object
quantity      46082 non-null int64
cook_time     46082 non-null object
dtypes: datetime64[ns](2), int64(2), object(2)
memory usage: 2.5+ MB


In [535]:
df_store.head(5)

,order_id,order_date,on_hand,product,quantity,cook_time
833,1644738,2021-06-30 21:40:00,2021-06-30 21:58:00,Limonada,1,17:52
834,1644738,2021-06-30 21:40:00,2021-06-30 21:58:00,Pancakes,1,17:52
835,1644738,2021-06-30 21:40:00,2021-06-30 21:58:00,No Cubiertos,1,17:52
836,1644738,2021-06-30 21:40:00,2021-06-30 21:58:00,Porción Hash Browns,1,17:52
837,1644738,2021-06-30 21:40:00,2021-06-30 21:58:00,Egg Bacon & Cheese Croissant,1,17:52


# Creacion de tabla pivote con muestra de datos

In [481]:
df_15 = df_store[:20]

In [482]:
df_15

,order_id,order_date,on_hand,product,quantity,cook_time
833,1644738,2021-06-30 21:40:00,2021-06-30 21:58:00,Limonada,1,17:52
834,1644738,2021-06-30 21:40:00,2021-06-30 21:58:00,Pancakes,1,17:52
835,1644738,2021-06-30 21:40:00,2021-06-30 21:58:00,No Cubiertos,1,17:52
836,1644738,2021-06-30 21:40:00,2021-06-30 21:58:00,Porción Hash Browns,1,17:52
837,1644738,2021-06-30 21:40:00,2021-06-30 21:58:00,Egg Bacon & Cheese Croissant,1,17:52
984,1645052,2021-07-01 08:49:00,2021-07-01 09:06:00,Arma tu Breakfast Bowl,1,16:36
985,1645052,2021-07-01 08:49:00,2021-07-01 09:06:00,Banano,1,16:36
986,1645052,2021-07-01 08:49:00,2021-07-01 09:06:00,Fresas,1,16:36
987,1645052,2021-07-01 08:49:00,2021-07-01 09:06:00,Açai,1,16:36
988,1645052,2021-07-01 08:49:00,2021-07-01 09:06:00,Hash Browns,1,16:36


In [483]:
df_15.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20 entries, 833 to 2157
Data columns (total 6 columns):
order_id      20 non-null int64
order_date    20 non-null datetime64[ns]
on_hand       20 non-null datetime64[ns]
product       20 non-null object
quantity      20 non-null int64
cook_time     20 non-null object
dtypes: datetime64[ns](2), int64(2), object(2)
memory usage: 1.1+ KB


### Creamos los dummies para productos

In [484]:
df_15_X = df_15.pivot_table(index='order_id', columns='product', values='quantity', aggfunc='sum')
df_15_X = df_15_X.fillna(0)
df_15_X

product,Arma tu Breakfast Bowl,Arándanos,Açai,Banano,Croissant de Almendra,Egg Bacon & Cheese Croissant,Fresas,Hash Browns,Huevos Revueltos,Jamón,Jugo de Naranja,Limonada,Mantequilla de Maní,No Cubiertos,Pancakes,Porción Hash Browns,Si Cubiertos
order_id,,,,,,,,,,,,,,,,,
1644738,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0
1645052,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
1647842,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1648102,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,1.0,0.0,0.0,0.0


In [485]:
type(df_15_X)
df_15_X.shape

(4, 17)

### Agrupamos por order_id

In [486]:
df_15_ = df_15.drop_duplicates(subset=['order_id'])
# df_15_.reset_index(drop=True)
df_15_y = df_15_[['order_id', 'order_date', 'on_hand','cook_time']]
print(df_15_y)
print(df_15_y.shape)

      order_id          order_date             on_hand cook_time
833    1644738 2021-06-30 21:40:00 2021-06-30 21:58:00     17:52
984    1645052 2021-07-01 08:49:00 2021-07-01 09:06:00     16:36
1999   1647842 2021-07-02 09:06:00 2021-07-02 09:29:00     22:23
2155   1648102 2021-07-02 10:41:00 2021-07-02 11:00:00     19:10
(4, 4)


### Ajustamos la columna salida a minutos

In [487]:
# Funcion determinar segundos o minutos

def cook_time_to_seconds(time):
    '''
    :param time: Tiempo en formato %M:%S
    :return total_seconds: Tiempo en segundos
    '''
    minutes = int(time[0:2])
    seconds = int(time[3:5])
    total_seconds = minutes * 60 + seconds 
    return total_seconds

def cook_time_to_minutes(time):
    '''
    :param time: Tiempo en formato %M:%S
    :return total_seconds: Tiempo en minutos en formato decimal
    '''
    minutes = int(time[0:2])
    seconds = int(time[3:5])
    total_minutes = minutes + (seconds/60)
    return total_minutes


In [488]:
df_15_y['cook_time_minutes'] = df_15_y['cook_time'].apply(lambda X: cook_time_to_minutes(str(X)))
#df_15_y['cook_time_sec'] = df_15_y['cook_time'].total_seconds()
df_15_y

<ipython-input-488-5af1be11a47a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_15_y['cook_time_minutes'] = df_15_y['cook_time'].apply(lambda X: cook_time_to_minutes(str(X)))


,order_id,order_date,on_hand,cook_time,cook_time_minutes
833,1644738,2021-06-30 21:40:00,2021-06-30 21:58:00,17:52,17.866667
984,1645052,2021-07-01 08:49:00,2021-07-01 09:06:00,16:36,16.600000
1999,1647842,2021-07-02 09:06:00,2021-07-02 09:29:00,22:23,22.383333
2155,1648102,2021-07-02 10:41:00,2021-07-02 11:00:00,19:10,19.166667


### Combinamos Entradas y Salida

In [385]:
df_final_15 = pd.merge(df_15_X, df_15_y, on='order_id', how='left')
#df_final_15 = df_final_15.drop(['order_id'], axis=1)
df_final_15

,order_id,Arma tu Breakfast Bowl,Arándanos,Açai,Banano,Croissant de Almendra,Egg Bacon & Cheese Croissant,Fresas,Hash Browns,Huevos Revueltos,Jamón,Jugo de Naranja,Limonada,Mantequilla de Maní,No Cubiertos,Pancakes,Porción Hash Browns,Si Cubiertos,order_date,on_hand,cook_time,cook_time_minutes
0,1644738,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,2021-06-30 21:40:00,2021-06-30 21:58:00,17:52,17.866667
1,1645052,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,2021-07-01 08:49:00,2021-07-01 09:06:00,16:36,16.600000
2,1647842,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2021-07-02 09:06:00,2021-07-02 09:29:00,22:23,22.383333
3,1648102,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,1.0,0.0,0.0,0.0,2021-07-02 10:41:00,2021-07-02 11:00:00,19:10,19.166667


In [266]:
df_final_15.shape

(4, 20)

# Creacion de tabla pivote con base de completa

In [536]:
# Funcion determinar segundos o minutos

def cook_time_to_seconds(time):
    '''
    :param time: Tiempo en formato %M:%S
    :return total_seconds: Tiempo en segundos
    '''
    minutes = int(time[0:2])
    seconds = int(time[3:5])
    total_seconds = minutes * 60 + seconds 
    return total_seconds

def cook_time_to_minutes(time):
    '''
    :param time: Tiempo en formato %M:%S
    :return total_seconds: Tiempo en minutos en formato decimal
    '''
    minutes = int(time[0:2])
    seconds = int(time[3:5])
    total_minutes = minutes + (seconds/60)
    return total_minutes


In [538]:
# Dummies para productos
print('Creando dummies para productos')
df_X = df_store.pivot_table(index='order_id', columns='product', values='quantity', aggfunc='sum')
df_X = df_X.fillna(0)
#df_pivot

# Agrupamos por order_id el dataframe inicial
print('organizando datos objetivo')
print('   * agrupando por Numero de orden')
df_ = df_store.drop_duplicates(subset=['order_id'])
# df_15_.reset_index(drop=True)
print('   * seleccionando variables')
df_y = df_[['order_id', 'order_date', 'on_hand','cook_time']]

# Creando la columna Minutos
print('   * Creando la columna de segundos')
df_y['cook_time_minutes'] = df_y['cook_time'].apply(lambda X: cook_time_to_minutes(str(X)))
print(f'El numero de resgistros es {df_X.shape[0]}')
print()
# Combinando entradas y salidas
print('**********')
print('Combinando X y y')
df_final = pd.merge(df_X, df_y, on='order_id', how='left')
#df_final = df_final.drop(['order_id'], axis=1)
df_final


Creando dummies para productos
organizando datos objetivo
   * agrupando por Numero de orden
   * seleccionando variables
   * Creando la columna de segundos
El numero de resgistros es 7941

**********
Combinando X y y


<ipython-input-538-dd878d26a73d>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_y['cook_time_minutes'] = df_y['cook_time'].apply(lambda X: cook_time_to_minutes(str(X)))


,order_id,Almendras,Almendras Fileteadas,Americano,Arequipe,Arma tu Breakfast Bowl,Arma tu Ensalada de Frutas,Arándanos,Arándanos Ensalada,Avocado Toast,Azucar,Azúcar,Açai,B&M Açai Bowl,Banano,Brownie,Brunch Californiano,Brunch Para Papá,Brunch is Love,Brunch para Papá + Happy Socks,Budweiser 315 ml,Cappuccino,Cappuccino de la Casa Entero,Cebolla,Chia,Chía,Cinnamon Roll,Club Colombia Rubia 330 ml,Coca Cola Sabor Original 300 ml,Coca Cola Sin Azúcar 300 ml,Coca-Cola Sabor Original 300 ml,Coca-Cola Sin Azúcar 300 ml,Coco Ensalada,Coco Rallado,Coco rallado,Combo Anti-Gripa,Corona 350 ml,Croissant De Almendra,Croissant de Almendra,Egg & Bacon Muffin,Egg Bacon & Cheese Croissant,Egg Benedict Muffin,"Egg, Bacon & Cheese Croissant",Espinaca,French Toast,Fresa Ensalada,Fresas,Galleta Chips de Chocolate,Granola,Green Fix,Grilled Cheese Sandwich,Ham Cheese & Egg Croissant,"Ham, Cheese & Egg Croissant",Hash Browns,Huevos Fritos,Huevos Revueltos,Jamón,Jugo de Mandarina,Jugo de Naranja,Jugo de mandarina,Jugo de naranja,Kiwi Ensalada,Latte,Latte de la Casa Entero,Leche Almendras,Leche Deslactosada,Leche Entera,Limonada,Manantial Con Gas 300 ml,Manantial Sin Gas 500 ml,Manantial sin Gas 500 ml,Mango Ensalada,Mantequilla de Maní,Manzana Verde Ensalada,Maíz,Mega Croissant,Melón Ensalada,Mini Croissant,No Cubiertos,"No, Cubiertos",Nutella,Palito De Queso,Pancakes,Pancakes con Fresas,Pancakes de Avena,Papaya Ensalada,Peanut Butter,Piña Ensalada,Porción Hash Browns,Porción de Fruta,Power Boost,Pura Vida,Queso Mozzarella,Queso Parmesano,Root Recovery,Salsa de Frutos Rojos,Sandía Ensalada,Shot Inmunidad,Shot Ultra Defensas,Shot energía,Si Cubiertos,"Si, Cubiertos","Si, porción de hash browns",Splenda,Spooky Brunch,Sprite 300 ml,Tocineta,Tomate,Turkey & Avocado Egg Muffin,Veggie Egg Muffin,Vitamin Boost,Waffles con fresas,Yogurt,order_date,on_hand,cook_time,cook_time_minutes
0,1590954,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2021-06-09 18:51:00,2021-06-09 19:29:00,38:14,38.233333
1,1591099,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2021-06-09 20:06:00,2021-06-09 20:29:00,23:07,23.116667
2,1591134,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2021-06-09 20:20:00,2021-06-09 20:39:00,19:06,19.100000
3,1591192,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2021-06-09 20:56:00,2021-06-09 21:16:00,19:13,19.216667
4,1591322,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0

In [539]:
df_final.to_csv('df_brunch_munch_medellin.csv')

# Cargamos datos almacenados previamente

In [592]:
df_store = pd.read_csv('df_brunch_munch_medellin.csv')
df_store.shape

(7941, 118)

## Cracion carateristicas de tiempo

In [593]:
# Create new features(year, month, day) from start and end time of data   
#df_store['YearPS'] = pd.to_datetime(df_store['order_date']).dt.year
#df_store['YearPE'] = pd.to_datetime(df_store['on_hand']).dt.year

df_store['MonthPS'] = pd.to_datetime(df_store['order_date']).dt.month
#df_store['MonthPE'] = pd.to_datetime(df_store['on_hand']).dt.month

df_store['DayPS'] = pd.to_datetime(df_store['order_date']).dt.day

df_store['HourPS'] = pd.to_datetime(df_store['order_date']).dt.hour
#df_store['HourPE'] = pd.to_datetime(df_store['on_hand']).dt.hour

df_store['MinutePS'] = pd.to_datetime(df_store['order_date']).dt.minute
#df_store['MinutePE'] = pd.to_datetime(df_store['on_hand']).dt.minute

# convert datetime column of order_date to just date
df_store['Date'] = pd.to_datetime(df_store['order_date']).dt.normalize()

In [594]:
df_store.sample(10)

,Unnamed: 0,order_id,Almendras,Almendras Fileteadas,Americano,Arequipe,Arma tu Breakfast Bowl,Arma tu Ensalada de Frutas,Arándanos,Arándanos Ensalada,Avocado Toast,Azucar,Azúcar,Açai,B&M Açai Bowl,Banano,Brownie,Brunch Californiano,Brunch Para Papá,Brunch is Love,Brunch para Papá + Happy Socks,Budweiser 315 ml,Cappuccino,Cappuccino de la Casa Entero,Cebolla,Chia,Chía,Cinnamon Roll,Club Colombia Rubia 330 ml,Coca Cola Sabor Original 300 ml,Coca Cola Sin Azúcar 300 ml,Coca-Cola Sabor Original 300 ml,Coca-Cola Sin Azúcar 300 ml,Coco Ensalada,Coco Rallado,Coco rallado,Combo Anti-Gripa,Corona 350 ml,Croissant De Almendra,Croissant de Almendra,Egg & Bacon Muffin,Egg Bacon & Cheese Croissant,Egg Benedict Muffin,"Egg, Bacon & Cheese Croissant",Espinaca,French Toast,Fresa Ensalada,Fresas,Galleta Chips de Chocolate,Granola,Green Fix,Grilled Cheese Sandwich,Ham Cheese & Egg Croissant,"Ham, Cheese & Egg Croissant",Hash Browns,Huevos Fritos,Huevos Revueltos,Jamón,Jugo de Mandarina,Jugo de Naranja,Jugo de mandarina,Jugo de naranja,Kiwi Ensalada,Latte,Latte de la Casa Entero,Leche Almendras,Leche Deslactosada,Leche Entera,Limonada,Manantial Con Gas 300 ml,Manantial Sin Gas 500 ml,Manantial sin Gas 500 ml,Mango Ensalada,Mantequilla de Maní,Manzana Verde Ensalada,Maíz,Mega Croissant,Melón Ensalada,Mini Croissant,No Cubiertos,"No, Cubiertos",Nutella,Palito De Queso,Pancakes,Pancakes con Fresas,Pancakes de Avena,Papaya Ensalada,Peanut Butter,Piña Ensalada,Porción Hash Browns,Porción de Fruta,Power Boost,Pura Vida,Queso Mozzarella,Queso Parmesano,Root Recovery,Salsa de Frutos Rojos,Sandía Ensalada,Shot Inmunidad,Shot Ultra Defensas,Shot energía,Si Cubiertos,"Si, Cubiertos","Si, porción de hash browns",Splenda,Spooky Brunch,Sprite 300 ml,Tocineta,Tomate,Turkey & Avocado Egg Muffin,Veggie Egg Muffin,Vitamin Boost,Waffles con fresas,Yogurt,order_date,on_hand,cook_time,cook_time_minutes,MonthPS,DayPS,HourPS,MinutePS,Date
220,220,1599612,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2021-06-14 07:03:00,2021-06-14 07:28:00,25:00,25.000000,6,14,7,3,2021-06-14
5258,5258,1861553,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2021-09-23 10:48:00,2021-09-23 11:04:00,16:23,16.383333,9,23,10,48,2021-09-23
4872,4872,1843583,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2021-09-17 07:54:00,2021-09-17 08:26:00,31:46,31.766667,9,17,7,54,2021-09-17
3116,3116,1752314,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,

In [595]:
# convert the column (it's a string) to datetime type
dt_series = pd.to_datetime(df_store['Date'])

# create datetime index passing the datetime series
dt_index = pd.DatetimeIndex(dt_series.values)

# 
df_new_store = df_store.set_index(dt_index)
df_new_store

,Unnamed: 0,order_id,Almendras,Almendras Fileteadas,Americano,Arequipe,Arma tu Breakfast Bowl,Arma tu Ensalada de Frutas,Arándanos,Arándanos Ensalada,Avocado Toast,Azucar,Azúcar,Açai,B&M Açai Bowl,Banano,Brownie,Brunch Californiano,Brunch Para Papá,Brunch is Love,Brunch para Papá + Happy Socks,Budweiser 315 ml,Cappuccino,Cappuccino de la Casa Entero,Cebolla,Chia,Chía,Cinnamon Roll,Club Colombia Rubia 330 ml,Coca Cola Sabor Original 300 ml,Coca Cola Sin Azúcar 300 ml,Coca-Cola Sabor Original 300 ml,Coca-Cola Sin Azúcar 300 ml,Coco Ensalada,Coco Rallado,Coco rallado,Combo Anti-Gripa,Corona 350 ml,Croissant De Almendra,Croissant de Almendra,Egg & Bacon Muffin,Egg Bacon & Cheese Croissant,Egg Benedict Muffin,"Egg, Bacon & Cheese Croissant",Espinaca,French Toast,Fresa Ensalada,Fresas,Galleta Chips de Chocolate,Granola,Green Fix,Grilled Cheese Sandwich,Ham Cheese & Egg Croissant,"Ham, Cheese & Egg Croissant",Hash Browns,Huevos Fritos,Huevos Revueltos,Jamón,Jugo de Mandarina,Jugo de Naranja,Jugo de mandarina,Jugo de naranja,Kiwi Ensalada,Latte,Latte de la Casa Entero,Leche Almendras,Leche Deslactosada,Leche Entera,Limonada,Manantial Con Gas 300 ml,Manantial Sin Gas 500 ml,Manantial sin Gas 500 ml,Mango Ensalada,Mantequilla de Maní,Manzana Verde Ensalada,Maíz,Mega Croissant,Melón Ensalada,Mini Croissant,No Cubiertos,"No, Cubiertos",Nutella,Palito De Queso,Pancakes,Pancakes con Fresas,Pancakes de Avena,Papaya Ensalada,Peanut Butter,Piña Ensalada,Porción Hash Browns,Porción de Fruta,Power Boost,Pura Vida,Queso Mozzarella,Queso Parmesano,Root Recovery,Salsa de Frutos Rojos,Sandía Ensalada,Shot Inmunidad,Shot Ultra Defensas,Shot energía,Si Cubiertos,"Si, Cubiertos","Si, porción de hash browns",Splenda,Spooky Brunch,Sprite 300 ml,Tocineta,Tomate,Turkey & Avocado Egg Muffin,Veggie Egg Muffin,Vitamin Boost,Waffles con fresas,Yogurt,order_date,on_hand,cook_time,cook_time_minutes,MonthPS,DayPS,HourPS,MinutePS,Date
2021-06-09,0,1590954,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2021-06-09 18:51:00,2021-06-09 19:29:00,38:14,38.233333,6,9,18,51,2021-06-09
2021-06-09,1,1591099,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2021-06-09 20:06:00,2021-06-09 20:29:00,23:07,23.116667,6,9,20,6,2021-06-09
2021-06-09,2,1591134,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2021-06-09 20:20:00,2021-06-09 20:39:00,19:06,19.100000,6,9,20,20,2021-06-09
2021-06-09,3,1591192,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0

### Agragamos la media diaria de tiempo de cocina

In [596]:
df_new_store['daily_time'] = df_new_store.reset_index().groupby(pd.Grouper(key='Date', freq='1D'))['cook_time_minutes'].mean()

In [597]:
df_new_store.head(1)

,Unnamed: 0,order_id,Almendras,Almendras Fileteadas,Americano,Arequipe,Arma tu Breakfast Bowl,Arma tu Ensalada de Frutas,Arándanos,Arándanos Ensalada,Avocado Toast,Azucar,Azúcar,Açai,B&M Açai Bowl,Banano,Brownie,Brunch Californiano,Brunch Para Papá,Brunch is Love,Brunch para Papá + Happy Socks,Budweiser 315 ml,Cappuccino,Cappuccino de la Casa Entero,Cebolla,Chia,Chía,Cinnamon Roll,Club Colombia Rubia 330 ml,Coca Cola Sabor Original 300 ml,Coca Cola Sin Azúcar 300 ml,Coca-Cola Sabor Original 300 ml,Coca-Cola Sin Azúcar 300 ml,Coco Ensalada,Coco Rallado,Coco rallado,Combo Anti-Gripa,Corona 350 ml,Croissant De Almendra,Croissant de Almendra,Egg & Bacon Muffin,Egg Bacon & Cheese Croissant,Egg Benedict Muffin,"Egg, Bacon & Cheese Croissant",Espinaca,French Toast,Fresa Ensalada,Fresas,Galleta Chips de Chocolate,Granola,Green Fix,Grilled Cheese Sandwich,Ham Cheese & Egg Croissant,"Ham, Cheese & Egg Croissant",Hash Browns,Huevos Fritos,Huevos Revueltos,Jamón,Jugo de Mandarina,Jugo de Naranja,Jugo de mandarina,Jugo de naranja,Kiwi Ensalada,Latte,Latte de la Casa Entero,Leche Almendras,Leche Deslactosada,Leche Entera,Limonada,Manantial Con Gas 300 ml,Manantial Sin Gas 500 ml,Manantial sin Gas 500 ml,Mango Ensalada,Mantequilla de Maní,Manzana Verde Ensalada,Maíz,Mega Croissant,Melón Ensalada,Mini Croissant,No Cubiertos,"No, Cubiertos",Nutella,Palito De Queso,Pancakes,Pancakes con Fresas,Pancakes de Avena,Papaya Ensalada,Peanut Butter,Piña Ensalada,Porción Hash Browns,Porción de Fruta,Power Boost,Pura Vida,Queso Mozzarella,Queso Parmesano,Root Recovery,Salsa de Frutos Rojos,Sandía Ensalada,Shot Inmunidad,Shot Ultra Defensas,Shot energía,Si Cubiertos,"Si, Cubiertos","Si, porción de hash browns",Splenda,Spooky Brunch,Sprite 300 ml,Tocineta,Tomate,Turkey & Avocado Egg Muffin,Veggie Egg Muffin,Vitamin Boost,Waffles con fresas,Yogurt,order_date,on_hand,cook_time,cook_time_minutes,MonthPS,DayPS,HourPS,MinutePS,Date,daily_time
2021-06-09,0,1590954,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2021-06-09 18:51:00,2021-06-09 19:29:00,38:14,38.233333,6,9,18,51,2021-06-09,24.916667


### Agregamos la cantidad de productos de la orden

In [598]:
df_new_store['product_quantity'] = df_new_store.iloc[:,2:113].sum(axis=1)

In [599]:
df_new_store.head(6)

,Unnamed: 0,order_id,Almendras,Almendras Fileteadas,Americano,Arequipe,Arma tu Breakfast Bowl,Arma tu Ensalada de Frutas,Arándanos,Arándanos Ensalada,Avocado Toast,Azucar,Azúcar,Açai,B&M Açai Bowl,Banano,Brownie,Brunch Californiano,Brunch Para Papá,Brunch is Love,Brunch para Papá + Happy Socks,Budweiser 315 ml,Cappuccino,Cappuccino de la Casa Entero,Cebolla,Chia,Chía,Cinnamon Roll,Club Colombia Rubia 330 ml,Coca Cola Sabor Original 300 ml,Coca Cola Sin Azúcar 300 ml,Coca-Cola Sabor Original 300 ml,Coca-Cola Sin Azúcar 300 ml,Coco Ensalada,Coco Rallado,Coco rallado,Combo Anti-Gripa,Corona 350 ml,Croissant De Almendra,Croissant de Almendra,Egg & Bacon Muffin,Egg Bacon & Cheese Croissant,Egg Benedict Muffin,"Egg, Bacon & Cheese Croissant",Espinaca,French Toast,Fresa Ensalada,Fresas,Galleta Chips de Chocolate,Granola,Green Fix,Grilled Cheese Sandwich,Ham Cheese & Egg Croissant,"Ham, Cheese & Egg Croissant",Hash Browns,Huevos Fritos,Huevos Revueltos,Jamón,Jugo de Mandarina,Jugo de Naranja,Jugo de mandarina,Jugo de naranja,Kiwi Ensalada,Latte,Latte de la Casa Entero,Leche Almendras,Leche Deslactosada,Leche Entera,Limonada,Manantial Con Gas 300 ml,Manantial Sin Gas 500 ml,Manantial sin Gas 500 ml,Mango Ensalada,Mantequilla de Maní,Manzana Verde Ensalada,Maíz,Mega Croissant,Melón Ensalada,Mini Croissant,No Cubiertos,"No, Cubiertos",Nutella,Palito De Queso,Pancakes,Pancakes con Fresas,Pancakes de Avena,Papaya Ensalada,Peanut Butter,Piña Ensalada,Porción Hash Browns,Porción de Fruta,Power Boost,Pura Vida,Queso Mozzarella,Queso Parmesano,Root Recovery,Salsa de Frutos Rojos,Sandía Ensalada,Shot Inmunidad,Shot Ultra Defensas,Shot energía,Si Cubiertos,"Si, Cubiertos","Si, porción de hash browns",Splenda,Spooky Brunch,Sprite 300 ml,Tocineta,Tomate,Turkey & Avocado Egg Muffin,Veggie Egg Muffin,Vitamin Boost,Waffles con fresas,Yogurt,order_date,on_hand,cook_time,cook_time_minutes,MonthPS,DayPS,HourPS,MinutePS,Date,daily_time,product_quantity
2021-06-09,0,1590954,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2021-06-09 18:51:00,2021-06-09 19:29:00,38:14,38.233333,6,9,18,51,2021-06-09,24.916667,4.0
2021-06-09,1,1591099,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2021-06-09 20:06:00,2021-06-09 20:29:00,23:07,23.116667,6,9,20,6,2021-06-09,24.916667,6.0
2021-06-09,2,1591134,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2021-06-09 20:20:00,2021-06-09 20:39:00,19:06,19.100000,6,9,20,20,2021-06-09,24.916667,4.0
2021-06-09,3,1591192,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0

Agregamos la media de productos de una orden al dia

In [600]:
df_new_store['daily_products_mean'] = df_new_store.reset_index().groupby(pd.Grouper(key='Date', freq='1D'))['product_quantity'].mean()

In [601]:
df_new_store.head(6)

,Unnamed: 0,order_id,Almendras,Almendras Fileteadas,Americano,Arequipe,Arma tu Breakfast Bowl,Arma tu Ensalada de Frutas,Arándanos,Arándanos Ensalada,Avocado Toast,Azucar,Azúcar,Açai,B&M Açai Bowl,Banano,Brownie,Brunch Californiano,Brunch Para Papá,Brunch is Love,Brunch para Papá + Happy Socks,Budweiser 315 ml,Cappuccino,Cappuccino de la Casa Entero,Cebolla,Chia,Chía,Cinnamon Roll,Club Colombia Rubia 330 ml,Coca Cola Sabor Original 300 ml,Coca Cola Sin Azúcar 300 ml,Coca-Cola Sabor Original 300 ml,Coca-Cola Sin Azúcar 300 ml,Coco Ensalada,Coco Rallado,Coco rallado,Combo Anti-Gripa,Corona 350 ml,Croissant De Almendra,Croissant de Almendra,Egg & Bacon Muffin,Egg Bacon & Cheese Croissant,Egg Benedict Muffin,"Egg, Bacon & Cheese Croissant",Espinaca,French Toast,Fresa Ensalada,Fresas,Galleta Chips de Chocolate,Granola,Green Fix,Grilled Cheese Sandwich,Ham Cheese & Egg Croissant,"Ham, Cheese & Egg Croissant",Hash Browns,Huevos Fritos,Huevos Revueltos,Jamón,Jugo de Mandarina,Jugo de Naranja,Jugo de mandarina,Jugo de naranja,Kiwi Ensalada,Latte,Latte de la Casa Entero,Leche Almendras,Leche Deslactosada,Leche Entera,Limonada,Manantial Con Gas 300 ml,Manantial Sin Gas 500 ml,Manantial sin Gas 500 ml,Mango Ensalada,Mantequilla de Maní,Manzana Verde Ensalada,Maíz,Mega Croissant,Melón Ensalada,Mini Croissant,No Cubiertos,"No, Cubiertos",Nutella,Palito De Queso,Pancakes,Pancakes con Fresas,Pancakes de Avena,Papaya Ensalada,Peanut Butter,Piña Ensalada,Porción Hash Browns,Porción de Fruta,Power Boost,Pura Vida,Queso Mozzarella,Queso Parmesano,Root Recovery,Salsa de Frutos Rojos,Sandía Ensalada,Shot Inmunidad,Shot Ultra Defensas,Shot energía,Si Cubiertos,"Si, Cubiertos","Si, porción de hash browns",Splenda,Spooky Brunch,Sprite 300 ml,Tocineta,Tomate,Turkey & Avocado Egg Muffin,Veggie Egg Muffin,Vitamin Boost,Waffles con fresas,Yogurt,order_date,on_hand,cook_time,cook_time_minutes,MonthPS,DayPS,HourPS,MinutePS,Date,daily_time,product_quantity,daily_products_mean
2021-06-09,0,1590954,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2021-06-09 18:51:00,2021-06-09 19:29:00,38:14,38.233333,6,9,18,51,2021-06-09,24.916667,4.0,4.500000
2021-06-09,1,1591099,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2021-06-09 20:06:00,2021-06-09 20:29:00,23:07,23.116667,6,9,20,6,2021-06-09,24.916667,6.0,4.500000
2021-06-09,2,1591134,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2021-06-09 20:20:00,2021-06-09 20:39:00,19:06,19.100000,6,9,20,20,2021-06-09,24.916667,4.0,4.500000
2021-06-09,3,1591192,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.

### Seleccionamos las variables de interes

In [602]:
# select features of interest for model
df_brunch_munch_medellin_final = df_new_store.drop(['cook_time','Unnamed: 0', 'order_id', 'order_date', 'on_hand'], axis=1)
df_brunch_munch_medellin_final

,Almendras,Almendras Fileteadas,Americano,Arequipe,Arma tu Breakfast Bowl,Arma tu Ensalada de Frutas,Arándanos,Arándanos Ensalada,Avocado Toast,Azucar,Azúcar,Açai,B&M Açai Bowl,Banano,Brownie,Brunch Californiano,Brunch Para Papá,Brunch is Love,Brunch para Papá + Happy Socks,Budweiser 315 ml,Cappuccino,Cappuccino de la Casa Entero,Cebolla,Chia,Chía,Cinnamon Roll,Club Colombia Rubia 330 ml,Coca Cola Sabor Original 300 ml,Coca Cola Sin Azúcar 300 ml,Coca-Cola Sabor Original 300 ml,Coca-Cola Sin Azúcar 300 ml,Coco Ensalada,Coco Rallado,Coco rallado,Combo Anti-Gripa,Corona 350 ml,Croissant De Almendra,Croissant de Almendra,Egg & Bacon Muffin,Egg Bacon & Cheese Croissant,Egg Benedict Muffin,"Egg, Bacon & Cheese Croissant",Espinaca,French Toast,Fresa Ensalada,Fresas,Galleta Chips de Chocolate,Granola,Green Fix,Grilled Cheese Sandwich,Ham Cheese & Egg Croissant,"Ham, Cheese & Egg Croissant",Hash Browns,Huevos Fritos,Huevos Revueltos,Jamón,Jugo de Mandarina,Jugo de Naranja,Jugo de mandarina,Jugo de naranja,Kiwi Ensalada,Latte,Latte de la Casa Entero,Leche Almendras,Leche Deslactosada,Leche Entera,Limonada,Manantial Con Gas 300 ml,Manantial Sin Gas 500 ml,Manantial sin Gas 500 ml,Mango Ensalada,Mantequilla de Maní,Manzana Verde Ensalada,Maíz,Mega Croissant,Melón Ensalada,Mini Croissant,No Cubiertos,"No, Cubiertos",Nutella,Palito De Queso,Pancakes,Pancakes con Fresas,Pancakes de Avena,Papaya Ensalada,Peanut Butter,Piña Ensalada,Porción Hash Browns,Porción de Fruta,Power Boost,Pura Vida,Queso Mozzarella,Queso Parmesano,Root Recovery,Salsa de Frutos Rojos,Sandía Ensalada,Shot Inmunidad,Shot Ultra Defensas,Shot energía,Si Cubiertos,"Si, Cubiertos","Si, porción de hash browns",Splenda,Spooky Brunch,Sprite 300 ml,Tocineta,Tomate,Turkey & Avocado Egg Muffin,Veggie Egg Muffin,Vitamin Boost,Waffles con fresas,Yogurt,cook_time_minutes,MonthPS,DayPS,HourPS,MinutePS,Date,daily_time,product_quantity,daily_products_mean
2021-06-09,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,38.233333,6,9,18,51,2021-06-09,24.916667,4.0,4.500000
2021-06-09,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,23.116667,6,9,20,6,2021-06-09,24.916667,6.0,4.500000
2021-06-09,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,19.100000,6,9,20,20,2021-06-09,24.916667,4.0,4.500000
2021-06-09,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,19.216667,6,9,20,56,2021-06-09,24.916667,4.0,4.500000
2021-06-10,0.0,0.0,

In [603]:
df_brunch_munch_medellin_final.to_csv('../datasets/df_brunch_munch_medellin_final.csv')